## 1. Install Required Libraries
First, you need to install the required libraries:

In [1]:
# Install Hugging Face Transformers, Datasets, Accelerate, and PEFT libraries
!pip install transformers datasets accelerate
!pip install bitsandbytes
!pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 19.6 MB/s eta 0:00:

## 2. Load Your Dataset
You’ve already loaded the dataset. Let’s prepare it for training:

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import DataCollatorForLanguageModeling

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')


Using device: cuda


In [3]:
df = pd.read_excel('/content/AmalStage.xlsx')
df.head()

,Question,Answer,Tag,Reference
0,ما هو تعريف البيع وفقًا للفصل 564؟,البيع هو عقد تنتقل به ملكية شيء أو حق من أحد ا...,تعريف، البيع، عقد، ملكية، ثمن.,الفصل 564\nالبيع عقد تنتقل به ملكية شيء أو حق ...
1,ما الذي ينتقل في عقد البيع؟,تنتقل ملكية شيء أو حق.,عقد البيع، ملكية، شيء، حق.,الفصل 564\nالبيع عقد تنتقل به ملكية شيء أو حق ...
2,ما هو الالتزام المالي في عقد البيع؟,الثمن الذي يلتزم به المتعاقد.,التزام مالي، الثمن.,الفصل 564\nالبيع عقد تنتقل به ملكية شيء أو حق ...
3,متى يجري على بيع المريض في مرض موته حكم الفصل ...,يجري حكم الفصل 354 إذا كان البيع لوارث وظهر في...,بيع المريض، مرض الموت، الفصل 354، وارث، قصد ال...,الفصل 565\nبيع المريض في مرض موته يجري عليه حك...
4,ما هو دليل قصد المحاباة في بيع المريض لوارث؟,أن يبيع له بأقل من الثمن المتعارف بكثير أو يشت...,دليل، قصد المحاباة، أقل من الثمن المتعارف، أكث...,الفصل 565\nبيع المريض في مرض موته يجري عليه حك...


In [4]:
df = df.dropna(subset=['Question', 'Answer']).reset_index(drop=True)

# Alternatively, fill missing values with empty strings (if you prefer not to drop rows)
# df['Question'] = df['Question'].fillna('')
# df['Answer'] = df['Answer'].fillna('')

# Prepare the data
def prepare_data(row):
    instruction = str(row['Question'])
    output = str(row['Answer'])
    return {'instruction': instruction, 'output': output}

# Apply the function to each row
data = df.apply(prepare_data, axis=1).to_list()

# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data)


## 3. Tokenize the Data
You need to tokenize the data using the same tokenizer as the Falcon 7B model:

In [5]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)

# Load the model in 8-bit mode
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [9]:
tokenizer.pad_token = tokenizer.eos_token

In [10]:
# Prepare data for training
def generate_prompt(data_point):
    return f"سؤال: {data_point['instruction']}\nإجابة: {data_point['output']}"
def tokenize_function(examples):
    prompts = [
        f"سؤال: {instruction}\nإجابة: {output}"
        for instruction, output in zip(examples['instruction'], examples['output'])
    ]
    tokenized = tokenizer(prompts, truncation=True, padding='max_length', max_length=512)
    return tokenized


tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
)

Map:   0%|          | 0/774 [00:00<?, ? examples/s]

In [18]:
# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("fine-tuned-falcon-7b")
tokenizer.save_pretrained("fine-tuned-falcon-7b")

# Function to generate responses
def generate_response(question):
    prompt = f"سؤال: {question}\nإجابة:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        max_length=512,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        temperature=0.7,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,  # Ensure that the generate function knows how to pad
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("إجابة:")[-1].strip()
    return answer

In [15]:
question = " ما هي حقوق الزوجة في حالة الطلاق في تونس؟"

In [16]:
# Example usage
answer = generate_response(question)
print(f"{answer}")

في تونس، تُمنح الزوجة في حالة الطلاق حق الحصول على النفقة، التي تُدفع لها من قبل زوجها لضمان استمرارها في الحياة بكرامة. كما يُمكنها المطالبة بأموالها الخاصة التي كانت في عهدة زوجها قبل الطلاق، وحقها في الحضانة لطفلها أو أطفالها، مع مراعاة مصلحة الطفل. 



In [17]:
question = "لمن حقوق الكفاله للاطفال في حاله الطلاق في القانون التونسي ?"

answer = generate_response(question)
print(f"{answer}")

في القانون التونسي، يحق للأطفال منفصلي الوالدين الحصول على الرعاية والنفقة من كلا الوالدين.  تُحدد المحكمة حقوق الزيارة للأب والأم بناءً على مصلحة الطفل.  يُمنح الطفل حق التعبير عن رغبته في العيش مع والد معين. 

